Environment && Library

In [142]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

Configuration

In [143]:
augmentation = True
quant = True
exit = 4
if(int(exit) > 4 or int(exit) < 1):
    exit = 1
if(augmentation == True):
   tittle = "aug_" + str(exit)
elif(augmentation == False):
    tittle = "non_" + str(exit)
    
if(quant == True):
    model_path = "/home/x/Desktop/" + tittle + "_uint8.tflite"
elif(quant == False):
    model_path = "/home/x/Desktop/" + tittle + "_float32.tflite"

Dataset

In [144]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
if(quant == True):
    data_type = "uint8"
    x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(data_type), x_test.reshape(-1, 28 ,28, 1).astype(data_type)
elif(quant == False):
    data_type = "float32"
    x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(data_type) / 255.0, x_test.reshape(-1, 28 ,28, 1).astype(data_type) / 255.0
validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= data_type)
validation_gen.fit(x_test)
validation_generator = validation_gen.flow(x_test, y_test, batch_size= 1, shuffle=True, seed= 2, save_to_dir= None)

Detail

In [145]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path= model_path)
print(interpreter.get_signature_list())
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(str(input_details))
print(str(output_details))

{'serving_default': {'inputs': ['input'], 'outputs': ['exit_4']}}
[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([ 1, 28, 28,  1], dtype=int32), 'shape_signature': array([-1, 28, 28,  1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 22, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


Validation

In [146]:
predict_time, correct, epochs = 0, 0, 120
accuracy_list = np.arange(1, 121).astype(np.float32)
latency_list = np.arange(1, 121).astype(np.float32)
if(epochs > 0):
    for i in range(epochs):
        x,y =validation_generator.next()
        start_time = time.time()
        for j in range(len(x)):
            interpreter.set_tensor(input_details[0]['index'], x)
            interpreter.invoke()


            output_data = interpreter.get_tensor(output_details[0]['index'])    ### index of outputs 
            index = np.argmax(output_data[0])
            predict_time += (time.time() - start_time)

            print(i, "\t", y[0], "\t", index, "\t", [round(k,3) for k in output_data[0]])
            if(index == y[0]):
                correct += 1
            if(epochs < 5):
                plt.imshow(x[0])
                plt.show()
            accuracy_list[i] = correct / (i + 1)
            latency_list[i] = predict_time * 1000 / (i + 1)

    print("predict_time:", (predict_time * 1000 / epochs), "ms")
    print("accuracy:", (correct * 100 / epochs), "%")

0 	 8 	 8 	 [0, 0, 0, 0, 0, 0, 13, 0, 242, 0]
1 	 8 	 8 	 [1, 0, 1, 0, 0, 0, 3, 0, 250, 0]
2 	 7 	 7 	 [0, 1, 1, 0, 0, 0, 0, 254, 0, 0]
3 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4 	 9 	 9 	 [0, 0, 0, 0, 8, 0, 0, 0, 1, 246]
5 	 8 	 8 	 [2, 0, 23, 1, 0, 1, 3, 0, 226, 0]
6 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7 	 2 	 2 	 [0, 0, 255, 0, 0, 0, 0, 0, 0, 0]
8 	 3 	 3 	 [0, 0, 0, 253, 0, 2, 0, 0, 0, 0]
9 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10 	 1 	 1 	 [0, 255, 0, 0, 0, 0, 0, 0, 0, 0]
11 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
12 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13 	 8 	 8 	 [0, 0, 17, 5, 2, 0, 0, 2, 216, 15]
14 	 4 	 4 	 [0, 0, 0, 0, 255, 0, 0, 0, 0, 0]
15 	 1 	 1 	 [0, 255, 0, 0, 0, 0, 0, 0, 0, 0]
16 	 6 	 6 	 [0, 0, 0, 0, 0, 0, 255, 0, 0, 0]
17 	 2 	 2 	 [0, 0, 224, 29, 0, 0, 0, 2, 1, 0]
18 	 0 	 0 	 [255, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19 	 1 	 1 	 [0, 255, 0, 0, 0, 0, 0, 0, 0, 0]
20 	 7 	 7 	 [0, 0, 8, 0, 0, 0, 0, 248, 0, 0]
21 	 1 	 1 	 [0, 255, 0, 0, 0, 0, 0, 0,

Accuracy List

In [147]:
print("[", end= "")
for i in range(120):
    print(accuracy_list[i], end= "")
    if(i != 119):
        print(",",end= " ")
print("]")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.969697, 0.9705882, 0.9714286, 0.9722222, 0.972973, 0.9736842, 0.974359, 0.975, 0.9756098, 0.97619045, 0.9767442, 0.97727275, 0.95555556, 0.95652175, 0.9361702, 0.9375, 0.93877554, 0.94, 0.9411765, 0.9423077, 0.9433962, 0.9444444, 0.94545454, 0.9464286, 0.94736844, 0.9310345, 0.9322034, 0.93333334, 0.93442625, 0.91935486, 0.9206349, 0.921875, 0.9230769, 0.92424244, 0.92537314, 0.9264706, 0.92753625, 0.9285714, 0.92957747, 0.9305556, 0.9315069, 0.9324324, 0.92, 0.92105263, 0.9220779, 0.9230769, 0.9240506, 0.925, 0.9259259, 0.9268293, 0.92771083, 0.9285714, 0.92941177, 0.9186047, 0.9195402, 0.92045456, 0.92134833, 0.9222222, 0.9230769, 0.92391306, 0.9247312, 0.9255319, 0.9263158, 0.9166667, 0.91752577, 0.9183673, 0.9191919, 0.91, 0.9108911, 0.9117647, 0.9126214, 0.91346157, 0.9142857, 0.9150943, 0.90654206, 0.9074074, 0.9082569,

Latency List

In [148]:
print("[", end= "")
for i in range(120):
    print(latency_list[i], end= "")
    if(i != 119):
        print(",",end= " ")
print("]")

[0.14591217, 0.10406971, 0.0872612, 0.07861853, 0.073194504, 0.069618225, 0.06679126, 0.06458163, 0.06281005, 0.06146431, 0.060428273, 0.059564907, 0.059237845, 0.05882127, 0.058460236, 0.058054924, 0.057697296, 0.05732642, 0.05783533, 0.058555603, 0.059445698, 0.060157344, 0.06089003, 0.061502058, 0.06204605, 0.06258488, 0.06304847, 0.06347043, 0.06394551, 0.06430944, 0.06467296, 0.06503612, 0.06536281, 0.065894686, 0.06610326, 0.066260494, 0.066383466, 0.06652506, 0.06670218, 0.06688833, 0.06704214, 0.06723404, 0.06737266, 0.06754832, 0.06773737, 0.06790783, 0.06806597, 0.068714224, 0.06882512, 0.06892681, 0.068585075, 0.06824732, 0.067895316, 0.06756076, 0.06723404, 0.06690196, 0.06657316, 0.06635847, 0.06634098, 0.06606976, 0.06581916, 0.065561265, 0.065413736, 0.065125525, 0.064842515, 0.06455725, 0.064565174, 0.06429939, 0.064034395, 0.06381103, 0.06357381, 0.06333987, 0.06310581, 0.06286518, 0.06264051, 0.06244057, 0.06224273, 0.062068302, 0.061904326, 0.06186068, 0.06167977, 0.

Weight && Bias

In [149]:
all_tensor_details = interpreter.get_tensor_details()
for tensor_item in all_tensor_details:
  print("Weight %s:" % tensor_item["name"])
  print(interpreter.tensor(tensor_item["index"])())

Weight serving_default_input:0:
[[[[  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]
   [  0]